In [1]:
import sys
import os
import wandb
!wandb off
# wandb.login()

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
W&B offline, running your script from this directory will only write metadata locally.


In [2]:
!mamba create -n py312 -y
!source /opt/conda/bin/activate py312 && mamba install python=3.12.2 jupyter mamba -y

!sudo rm /opt/conda/bin/python3
!sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python3
!sudo rm /opt/conda/bin/python3.7
!sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python3.7
!sudo rm /opt/conda/bin/python
!sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.25.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: [

In [3]:
!git clone https://github.com/Ben-Gayther/predicting_goodreads_review_ratings.git

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Cloning into 'predicting_goodreads_review_ratings'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 277 (delta 147), reused 213 (delta 94), pack-reused 0
Receiving objects: 100% (277/277), 2.42 MiB | 6.78 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [4]:
os.chdir("predicting_goodreads_review_ratings")

In [5]:
yaml_contents = """
---
LOGGING:
  LEVEL: "INFO"
  FORMAT: "%(asctime)s - %(name)s - %(levelname)s - %(message)s"

TRAINING_ARGS:
  MODEL_NAME: "nlptown/bert-base-multilingual-uncased-sentiment"
  LEARNING_RATE: 2.0e-5
  MAX_LENGTH: 256
  BATCH_SIZE: 64
  EPOCHS: 1
  FULL_DATASET: True
  TEST_RUN: True
  OUTPUT_DIR: "models/"
  SUBMISSION_NAME: "/kaggle/working/submission.csv"

DATA_ARGS:
  INPUT_TRAIN_DATA: "/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv"
  OUTPUT_TRAIN_NAME: "data/processed_goodreads_train.csv"
  INPUT_TEST_DATA: "/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv"
  OUTPUT_TEST_NAME: "data/processed_goodreads_test.csv"
"""

In [6]:
with open("config.yaml", "w") as file:
    file.write(yaml_contents)

In [7]:
!pip install -r requirements.txt

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Ignoring colorama: markers 'python_version >= "3.12" and python_version < "4.0" and platform_system == "Windows"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.9/303.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
!python src/prepare_data.py

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2024-04-04 09:47:15,910 - root - INFO - Read data from /kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv, fields are Index(['user_id', 'book_id', 'review_id', 'rating', 'review_text',
       'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes',
       'n_comments'],
      dtype='object')
2024-04-04 09:47:40,184 - root - INFO - Preprocessed data
2024-04-04 09:49:08,639 - root - INFO - Saved data to data/processed_goodreads_train.csv
2024-04-04 09:49:20,555 - root - INFO - Read data from /kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv, fields are Index(['user_id', 'book_id', 'review_id', 'review_text', 'date_added',
       'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments'],
      dtype='object')
2024-04-04 09:49:33,635 - root - INFO - Preprocessed data
2024-04-04 09:50:20,684 - root - INFO - Saved data to data/processed_goodreads_test

In [9]:
!python src/train_model.py

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2024-04-04 09:50:48,347 - root - INFO - Doing test run with only 1000 samples
2024-04-04 09:50:48,349 - root - INFO - Read data from data/processed_goodreads_train.csv
tokenizer_config.json: 100%|██████████████████| 39.0/39.0 [00:00<00:00, 201kB/s]
config.json: 100%|█████████████████████████████| 953/953 [00:00<00:00, 5.10MB/s]
vocab.txt: 100%|█████████████████████████████| 872k/872k [00:00<00:00, 4.93MB/s]
special_tokens_map.json: 100%|██████████████████| 112/112 [00:00<00:00, 634kB/s]
2024-04-04 09:50:49,907 - root - INFO - Using nlptown/bert-base-multilingual-uncased-sentiment tokenizer and model
pytorch_model.bin: 100%|██████████████████████| 669M/669M [00:02<00:00, 288MB/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classif

In [10]:
!python src/make_preds.py

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2024-04-04 09:51:33,086 - root - INFO - Loaded model
2024-04-04 09:51:33,469 - root - INFO - Loaded tokenizer
2024-04-04 09:51:45,722 - root - INFO - Doing test run with only 100 samples
2024-04-04 09:51:45,722 - root - INFO - Loaded test data
Map: 100%|███████████████████████████| 100/100 [00:00<00:00, 1464.81 examples/s]
2024-04-04 09:51:45,831 - root - INFO - Tokenized test data
/opt/conda/envs/py312/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
100%